In [23]:
import json
import io
import gzip
import pandas as pd
import glob

bytes_io = io.BytesIO()
with open('out/tpugraphs_layout/run_4291fe6bee6416cd2571c65e3e4d5c8c.jsonz', 'rb') as f:
    data = gzip.GzipFile(fileobj=f)
    data = data.read()

In [3]:
import plotly.express as px

In [4]:
data_dict = json.loads(data)

In [5]:
results = pd.DataFrame(data_dict['train_curve'])

# Layout Performance

In [6]:
data_dict

{'train_curve': {'epoch': [0],
  'train_loss': [28.02408790588379],
  'train_opa': [0.5479363799095154],
  'val_loss': [27.880842208862305],
  'val_opa': [0.6327160596847534]},
 'final_opa': {},
 'args': {'source': 'xla',
  'search': 'random',
  'epochs': 10,
  'batch_size': 8,
  'configs': 16,
  'max_configs': 1000,
  'early_stop': 40,
  'keep_nodes': 5000,
  'learning_rate': 0.001,
  'clip_norm': 0.01,
  'out_dir': '~/out/tpugraphs_layout',
  'results_csv': '/root/out/tpugraphs_layout/results_1695136943079_xla_random.csv',
  'validate_batches': 10,
  'run_id': ''}}

In [7]:
layout_files = glob.glob('out/tpugraphs_layout/*.jsonz')

for file in layout_files:
    with open(file, 'rb') as f:
        data = gzip.GzipFile(fileobj=f)
        data = data.read()
        data_dict = json.loads(data)
        print(f'{data_dict["args"]["source"]}-{data_dict["args"]["search"]}: {max(data_dict["train_curve"]["val_opa"])}')
        print(f'{data_dict["args"]["epochs"]}')
        print(f'Completed {len(data_dict["train_curve"]["epoch"])}')
        print(f'{data_dict["args"]["batch_size"]}')
        print(f'{data_dict["args"]["configs"]}')
        print('='*15)

xla-default: 0.5612998604774475
10
Completed 3
4
16
xla-random: 0.6327160596847534
10
Completed 1
8
16
nlp-random: 0.5458333492279053
10
Completed 7
4
16
nlp-default: 0.543749988079071
10
Completed 7
4
16
xla-random: 0.5083333253860474
4
Completed 2
8
16
xla-random: 0.6620370149612427
10
Completed 6
4
16
nlp-random: 0.5318883061408997
10
Completed 3
4
16
xla-random: 0.5270833373069763
4
Completed 1
8
16
xla-random: 0.5979166626930237
10
Completed 4
8
16


In [8]:
tile_files = glob.glob('out/tpugraphs_tiles/*.jsonz')

for file in tile_files:
    with open(file, 'rb') as f:
        data = gzip.GzipFile(fileobj=f)
        data = data.read()
        data_dict = json.loads(data)
        print(f'{max(data_dict["train_curve"]["val_opa"])}')

0.8747217655181885


In [16]:
max(data_dict['train_curve']['val_opa'])

0.8747217655181885

In [14]:
data_dict['args']

{'epochs': 100,
 'eval_every': 5,
 'batch_size': 10,
 'configs': 10,
 'early_stop': 20,
 'losses': 'ListMLELoss:1,MSE:0.02',
 'learning_rate': 0.001,
 'clip_norm': 0.001,
 'model': 'EarlyJoinSAGE',
 'model_kwargs_json': '{}',
 'test_mode': 'predictions',
 'out_dir': '~/out/tpugraphs_tiles',
 'results_csv': '/root/out/tpugraphs_tiles/results_1695160398841.csv',
 'validate_batches': -1,
 'run_id': ''}

In [53]:
data_dict['final_error']

{'val': {'1': 0.1244023916184552,
  '5': 0.037590790343477766,
  '10': 0.023528117946076395}}

In [23]:
json.dumps({'Hello':'World'}).encode()

b'{"Hello": "World"}'

In [25]:
import tensorflow as tf

In [26]:
with tf.io.gfile.GFile('out/tpugraphs_layout/run_24846e5af61bce70662383fad92f1a53.jsonz', 'rb') as f:
    data = f.read()

# Tiles Beam Experiment

In [20]:
import tensorflow as tf

In [30]:
beam_tile_files = glob.glob('/root/out/tpugraphs_tiles/beam/*.jsonz')
max_val_opa = 0
max_file = ''
for file_name in beam_tile_files:
    with open(file_name, 'rb') as f:
        data = gzip.GzipFile(fileobj=f)
        data = data.read()
        data_dict = json.loads(data)
        local_ = max(data_dict['train_curve']['val_opa'])
        if local_ > max_val_opa:
            max_val_opa = local_
            max_file = file_name

In [31]:
max_val_opa

0.7622220516204834

In [32]:
max_file

'/root/out/tpugraphs_tiles/beam/run_fc37be71ce966509a5250595d4205455.jsonz'

In [33]:
with open(max_file, 'rb') as f:
        data = gzip.GzipFile(fileobj=f)
        data = data.read()
        data_dict = json.loads(data)

In [41]:
data_dict['args']

{'epochs': 500,
 'eval_every': 2,
 'batch_size': 10,
 'configs': 8,
 'early_stop': 10,
 'losses': 'ListMLELoss:1',
 'learning_rate': 0.001,
 'clip_norm': 0.001,
 'model': 'MLP',
 'model_kwargs_json': '{"op_embed_dim": 64, "hidden_dim": 64, "mlp_layers": 2}',
 'test_mode': 'metrics',
 'out_dir': '~/out/tpugraphs_tiles/beam',
 'results_csv': '/root/out/tpugraphs_tiles/beam/results_1695298181833.csv',
 'validate_batches': 50,
 'run_id': 0}

In [46]:
!python tiles_train.py \
    --batch=10 \
    --early_stop=20 \
    --lr=0.001 \
    --clip_norm=0.001 \
    --configs=8 \
    --model=MLP \
    --losses=ListMLELoss:1 \
    --model_kwargs_json='{"op_embed_dim": 64, "hidden_dim": 64, "mlp_layers": 2}' \
    

dataset cache file:  /root/data/tpugraphs/cache/tile/1b7724c4cac8d16759868d6e7c1c773d-cache.npz
loaded from /root/data/tpugraphs/cache/tile/1b7724c4cac8d16759868d6e7c1c773d-cache.npz
dataset cache file:  /root/data/tpugraphs/cache/tile/fcadc92f971864fbda69106d46a417d3-cache.npz
loaded from /root/data/tpugraphs/cache/tile/fcadc92f971864fbda69106d46a417d3-cache.npz
dataset cache file:  /root/data/tpugraphs/cache/tile/b615a0d1c22f18055a1761b9270a2bd6-cache.npz
loaded from /root/data/tpugraphs/cache/tile/b615a0d1c22f18055a1761b9270a2bd6-cache.npz
565/565 [==============================] - 5s 3ms/step - loss: 10.1320 - opa_metric: 0.7301 - val_loss: 9.7538 - val_opa_metric: 0.7447
I0921 15:41:00.956149 281473127567376 train_lib.py:152]  * [@0] Validation (NEW BEST): 0.7447004318237305
I0921 15:41:00.956754 281473127567376 train_lib.py:70] wrote /root/out/tpugraphs_tiles/run_e0e9d841871f951e2fb09ee450c4f3c5.jsonz
I0921 15:41:01.348593 281473127567376 signature_serialization.py:148] Function 

In [39]:
!python tiles_train.py --helpfull

Binary for invoking the training loop.
flags:

absl.app:
  -?,--[no]help: show this help
    (default: 'false')
  --[no]helpfull: show full help
    (default: 'false')
  --[no]helpshort: show this help
    (default: 'false')
  --[no]helpxml: like --helpfull, but generates XML output
    (default: 'false')
  --[no]only_check_args: Set to true to validate args and exit.
    (default: 'false')
  --[no]pdb: Alias for --pdb_post_mortem.
    (default: 'false')
  --[no]pdb_post_mortem: Set to true to handle uncaught exceptions with PDB post
    mortem.
    (default: 'false')
  --profile_file: Dump profile information to a file (for python -m pstats).
    Implies --run_with_profiling.
  --[no]run_with_pdb: Set to true for PDB debug mode
    (default: 'false')
  --[no]run_with_profiling: Set to true for profiling the script. Execution will
    be slower, and the output format might change over time.
    (default: 'false')
  --[no]use_cprofile_for_profiling: Use cProfile instead of the profile m